In [ ]:
# file paths
sanctions_path = input('Paste relative path of xlsx file')

In [ ]:
import pandas as pd
from datetime import datetime

def write_suspension_reason(sent_off, event_type, match_event_subtype):
  if sent_off == 'No' and event_type == 'Yellow card':
    return 'Yellow-card accumulation'
  elif sent_off == 'Yes' and event_type == 'Yellow card':
    return 'Second caution'
  else:
    return match_event_subtype
  
def rewrite_club(club, serving_team):
  if type(serving_team) is str:
    return serving_team
  else:
    return club
  
def adjust_for_plurals(value, is_match):
  if is_match:
    if value == 1:
      return 'match'
    else:
      return 'matches'
  else:
    if value == 1:
      return 'month'
    else:
      return 'months'
    
def rewrite_values(value, is_match, is_not_months_for_remaining, return_date):
    if is_not_months_for_remaining:
      return return_date
    else:
      return f'{value} {adjust_for_plurals(value, is_match)}'
    
def rewrite_remaining(value, is_match, return_date):
  if is_match:
    return f'{value} {adjust_for_plurals(value, is_match)}'
  else:
    return_timestamp = pd.to_datetime(return_date).date()
    return return_timestamp

In [ ]:
sanctions = pd.read_excel(sanctions_path)

sanctions = sanctions[sanctions['Competition type'].str.contains('L1 Cup|Ontario')]

sanctions = sanctions[sanctions['Case status'].str.contains('CONFIRMED')]

sanctions['Team'] = sanctions.apply(lambda x: rewrite_club(x['Club'], x['Serving team']), axis=1)

sanctions['Reason for Suspension'] = sanctions.apply(lambda x: write_suspension_reason(x['Sent off'], x['Event Type'], x['Match event sub-type']), axis=1)

sanctions['Reason for Suspension'] = sanctions['Reason for Suspension'].apply(lambda x: 'Denying an obvious goalscoring opportunity' if 'Denying an obvious goalscoring opportunity' in x else x)

sanctions['Sanction type'] = sanctions['Sanction type'].map(lambda x: True if x == 'Match suspension' else False)

sanctions['Value'] = sanctions.apply(lambda x: rewrite_values(x['Value'], x['Sanction type'], False, x['Sanction date to (EDT)']), axis=1)

sanctions['Remaining'] = sanctions.apply(lambda x: rewrite_remaining(x['Remaining'], x['Sanction type'], x['Sanction date to (EDT)']), axis=1)

sanctions = sanctions[['Club', 'Competition type', 'Person name', 'Offender type', 'Reason for Suspension', 'Value', 'Remaining']].sort_values(by=['Club'])

sanctions = sanctions.rename(columns={'Competition type': 'Competition', 'Person name': 'Name', 'Offender type': 'Person type', 'Value': 'Total suspended', 'Remaining': 'Total remaining'})

sanctions

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')

last_updated = pd.DataFrame({ 'Club': ['Last updated:'], 'Competition': [today]})

sanctions = pd.concat([sanctions, last_updated], ignore_index=True)

sanctions.to_csv(f'output/active_suspensions_{today}.csv', index=False)